In [1]:
# https://udemy.com/recommender-systems
# https://deeplearningcourses.com/recommender-systems
from __future__ import print_function, division
from builtins import range, input

import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

# carregando dados
import os
if not os.path.exists('user2movie.json') or \
   not os.path.exists('movie2user.json') or \
   not os.path.exists('usermovie2rating.json') or \
   not os.path.exists('usermovie2rating_test.json'):
   import preprocess2dict


with open('user2movie.json', 'rb') as f:
  user2movie = pickle.load(f)

with open('movie2user.json', 'rb') as f:
  movie2user = pickle.load(f)

with open('usermovie2rating.json', 'rb') as f:
  usermovie2rating = pickle.load(f)

with open('usermovie2rating_test.json', 'rb') as f:
  usermovie2rating_test = pickle.load(f)


In [2]:
N = np.max(list(user2movie.keys())) + 1

# filmes dos dados de teste podem não estar nos dados de treino
# para garantir que isso nao aconteca vamos pegar o maior id de filme de ambas as bases
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

#if N > 10000:
#  print("N =", N, "are you sure you want to continue?")
#  print("Comment out these lines if so...")
#  exit()



N: 10000 M: 2000


In [3]:
# user-user correlation --------------------
inicio = time.time()
# para achar as similaridades vamos realizar O(N^2 * M) calculos!
# o ideal é paralelizar esse processamento apartir de uma quantidade de dados significativa
# nota: na realidade vamos realizar apen
as metade destes calculos pois é uma matriz simétrica

K = 25 # numero de vizinhos para serem considerados similares
limit = 5 # quantidade minima de filmes que um usuário deve ter em comum com o outro para ser considerado similar
neighbors = [] # guardar os vizinhos do usuario
averages = [] # media de notas de cada usuário
deviations = [] # desvio das notas de cada usuário

for i in range(N):# achando os vizinhos para todos os usuários
  #filmes que o usuario deu nota  
  movies_i = user2movie[i]
  movies_i_set = set(movies_i)

  # calculando média e desvio
  ratings_i = { movie:usermovie2rating[(i, movie)] for movie in movies_i }#dicionario (usuario,filme:nota)
  avg_i = np.mean(list(ratings_i.values()))#media das notas do usuario
  dev_i = { movie:(rating - avg_i) for movie, rating in ratings_i.items() }#calculando o desvio do usuarios
  dev_i_values = np.array(list(dev_i.values()))# transformando em np array para calcular sigma(erro quadrático)
  sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))#calculando sigma

  # salvando media e desvio
  averages.append(avg_i)
  deviations.append(dev_i)
    
  #calculando o peso usuario vs usuario para cada usuario
  sl = SortedList()
  for j in range(N):
    if j != i:#nao comparar o usuario com ele mesmo
        
      #filmes que o vizinho deu nota 
      movies_j = user2movie[j]
      movies_j_set = set(movies_j)
      common_movies = (movies_i_set & movies_j_set) # filmes em comum
        
      if len(common_movies) > limit:#se tiver mais de x filmes em comum
        
        # calcula media e desvio do vizinho
        ratings_j = { movie:usermovie2rating[(j, movie)] for movie in movies_j }
        avg_j = np.mean(list(ratings_j.values()))
        dev_j = { movie:(rating - avg_j) for movie, rating in ratings_j.items() }
        dev_j_values = np.array(list(dev_j.values()))
        sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))

        # calculando o coeficinete de correlacao entre ambos
        numerator = sum(dev_i[m]*dev_j[m] for m in common_movies)
        w_ij = numerator / (sigma_i * sigma_j)

        # insere valores dentro da lista e os trunca(peso, id vizinho)
        # negativa o peso pois sorted list traz os itens em ordem ascendente(menor no topo)
        # porém queremos os vizinhos com MAIOR correlacao
        sl.add((-w_ij, j))
        if len(sl) > K:#limita o numero de vizinhos
          del sl[-1]

  # armazena os vizinhos
  neighbors.append(sl)

  # print out useful things
  if i % 100 == 0:
    print(i)
fim = time.time()
print(fim - inicio)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
61617.86864948273


In [5]:
# calculando erro médio quadratico de treino e teste usando os vizinhos

#predicao de nota do usuario+filme
def predict(i, m):
  # soma ponderada dos desvios para cada vizinho
  numerator = 0
  denominator = 0
  for neg_w, j in neighbors[i]:
    # lembrando que as notas foram multiplicadas por -1, devemos reverter isso
    try:
      numerator += -neg_w * deviations[j][m]
      denominator += abs(neg_w)
    except KeyError:
      # neighbor may not have rated the same movie
      # don't want to do dictionary lookup twice
      # so just throw exception
      pass

  if denominator == 0:#caso nao tenha nenhum vizinho com peso
    prediction = averages[i]
  else:
    prediction = numerator / denominator + averages[i]
  prediction = min(5, prediction)# max rating is 5
  prediction = max(0.5, prediction) # min rating is 0.5
  return prediction

#treino
train_predictions = []
train_targets = []
for (i, m), target in usermovie2rating.items():
  # calcula a predicao para cada usuario+filme
  prediction = predict(i, m)

  # salva o resultado e o resultado esperado
  train_predictions.append(prediction)
  train_targets.append(target)

# teste
test_predictions = []
test_targets = []
for (i, m), target in usermovie2rating_test.items():
  # calcula a predicao para cada usuario+filme
  prediction = predict(i, m)
  # salva o resultado e o resultado esperado
  test_predictions.append(prediction)
  test_targets.append(target)


# calculate accuracy
def mse(p, t):
  p = np.array(p)
  t = np.array(t)
  return np.mean((p - t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

train mse: 0.5533568819140385
test mse: 0.6412857695473799
